In [28]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.naive_bayes import GaussianNB,BernoulliNB,MultinomialNB
from sklearn.ensemble import VotingClassifier
from sklearn import cross_validation

In [29]:
url = "https://raw.githubusercontent.com/ga-students/SF-DAT-20/master/Data/bank.csv"
BankData = pd.read_csv(url)
BankData.head(5)

,age,marital,education,default,balance,housing,loan,contact,duration,campaign,pdays,previous,poutcome,y
0,30,married,primary,no,1787,no,no,cellular,79,1,-1,0,unknown,no
1,33,married,secondary,no,4789,yes,yes,cellular,220,1,339,4,failure,no
2,35,single,tertiary,no,1350,yes,no,cellular,185,1,330,1,failure,no
3,30,married,tertiary,no,1476,yes,yes,unknown,199,4,-1,0,unknown,no
4,59,married,secondary,no,0,yes,no,unknown,226,1,-1,0,unknown,no


In [30]:
BankData['y'].unique()  #you can use unique if you would like to find out how many unique attributes each variable have

array(['no', 'yes'], dtype=object)

For dictionary of data please refer to https://archive.ics.uci.edu/ml/datasets/Bank+Marketing

#### Our goal is to define a model best predicts outcome y - success of the marketing campaign. This is the very first a dataset we used to introduce Logistic Regression Models. This time we know few more tricks! In this lecture we are going to use as many techniques as possible on our dataset to make the least possible error! Goal is to use, Logistic Regression, Naive Bayes, Random Forest, KNN, and using VotingClassifier() to combine the results. 

In [31]:
Default_dummy  = pd.get_dummies(BankData['default'], prefix = 'default')
del Default_dummy['default_no']

marital_dummy  = pd.get_dummies(BankData['marital'], prefix = 'marital')
del marital_dummy['marital_married']
del marital_dummy['marital_divorced']

housing_dummy  = pd.get_dummies(BankData['housing'], prefix = 'housing')
del housing_dummy['housing_no']

loan_dummy = pd.get_dummies(BankData['loan'], prefix = 'loan')
del loan_dummy['loan_no']


BankData = pd.concat([BankData,marital_dummy , Default_dummy, housing_dummy, loan_dummy], axis=1)
BankData.head()

,age,marital,education,default,balance,housing,loan,contact,duration,campaign,pdays,previous,poutcome,y,marital_single,default_yes,housing_yes,loan_yes
0,30,married,primary,no,1787,no,no,cellular,79,1,-1,0,unknown,no,0,0,0,0
1,33,married,secondary,no,4789,yes,yes,cellular,220,1,339,4,failure,no,0,0,1,1
2,35,single,tertiary,no,1350,yes,no,cellular,185,1,330,1,failure,no,1,0,1,0
3,30,married,tertiary,no,1476,yes,yes,unknown,199,4,-1,0,unknown,no,0,0,1,1
4,59,married,secondary,no,0,yes,no,unknown,226,1,-1,0,unknown,no,0,0,1,0


In [32]:
X = BankData[['age','balance','duration','campaign','pdays','previous','marital_single','default_yes','housing_yes','loan_yes']].copy()
y = BankData['y']

In [33]:
X

,age,balance,duration,campaign,pdays,previous,marital_single,default_yes,housing_yes,loan_yes
0,30,1787,79,1,-1,0,0,0,0,0
1,33,4789,220,1,339,4,0,0,1,1
2,35,1350,185,1,330,1,1,0,1,0
3,30,1476,199,4,-1,0,0,0,1,1
4,59,0,226,1,-1,0,0,0,1,0
5,35,747,141,2,176,3,1,0,0,0
6,36,307,341,1,330,2,0,0,1,0
7,39,147,151,2,-1,0,0,0,1,0
8,41,221,57,2,-1,0,0,0,1,0
9,43,-88,313,1,147,2,0,0,1,1


In [24]:
clf1 = LogisticRegression()
clf2 = RandomForestClassifier(max_depth=5, n_estimators = 1000)
clf3 = BernoulliNB()
clf4 = MultinomialNB()
clf5 = GaussianNB()
clf6 = KNeighborsClassifier()
clf7 = GradientBoostingClassifier(learning_rate=0.01)

In [25]:
eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('bnb', clf3),('mnb',clf4),('gnb',clf5),('knn',clf6),('gbc',clf7)], voting='hard')

In [27]:
for clf in [clf1,clf2,clf3,clf4,clf5,clf6,clf7,eclf]:
    scores = cross_validation.cross_val_score(clf, X, y, cv=5, scoring='accuracy')
    print scores.mean()

0.889626216203
0.882770253753
0.876357258104
0.862423116413
0.873477484966
0.884760426343


ValueError: Input X must be non-negative